In [1]:
!pip install --upgrade --quiet gspread

In [2]:
#@title Imports
import pandas as pd
import numpy as np
import pprint
from google.colab import drive
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, SelectMultiple
from IPython.display import clear_output

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [7]:
file_path = "Product Data" #@param {type:"string"}

In [8]:
#@title DataFrame

worksheet = gc.open(file_path).sheet1

rows = worksheet.get_all_values()

df = pd.DataFrame.from_records(rows[1:],columns=rows[0])


pd.set_option('display.max_colwidth', None) 
pd.set_option("display.max_rows", None)

In [9]:
#@title Data Preview
df.head()

,Serial Number,Defect Status,Date,Time,Month,Started Month,Finished Month,Week,Week #,Operator,Operation,Station,Table,Location,Latitude,Gore,Type,Film Type,SubProcess,Condition,Criteria,Disposition,MRB Disposition,Comment
0,TK-778,Closed,2014-05-15,21:09:00,May 2014,May 2019,May 2019,20,3,103269,,3,3,B13_33A,33,13,PB,CONVENTIONAL,GORE SEALS,CONTAMINATION,CONTAMINATION IN THE GAS SEAL THAT VIOLATES THE MINIMUM SEAL REQUIREMENT (1/16 IN FROM GAS EDGE); IF REMOVABLE CLEAN AREA FIRST,VERTROD,,
1,TK-778,Closed,2014-05-15,21:43:00,May 2014,May 2019,May 2019,20,3,103269,,3,3,B16_35A,35,16,PB,CONVENTIONAL,GORE SEALS,CONTAMINATION,CONTAMINATION IN THE GAS SEAL THAT VIOLATES THE MINIMUM SEAL REQUIREMENT (1/16 IN FROM GAS EDGE); IF REMOVABLE CLEAN AREA FIRST,VERTROD,,
2,TK-778,Closed,2014-05-15,21:44:00,May 2014,May 2019,May 2019,20,3,103269,,3,3,B16_35A,35,16,PB,CONVENTIONAL,GORE SEALS,CLEAR EDGE,THINNING THAT VIOLATES THE MINIMUM SEAL REQUIREMENT (1/16 IN FROM GAS EDGE),VERTROD,,
3,TK-778,Closed,2014-05-15,21:54:00,May 2014,May 2019,May 2019,20,3,103269,,3,3,B18_31A,31,18,PB,CONVENTIONAL,GORE FILM,GEL,GELS WITH FILM THINNING,SSA PATCH,,
4,TK-778,Closed,2014-05-16,2:30:00,May 2014,May 2019,May 2019,20,3,102533,,3,3,B19_14B,-14,19,PB,CONVENTIONAL,GORE FILM,SCRATCH/ABRASION,IDENTIFIED ANYWHERE DURING/AFTER SEALING,SSA PATCH,,


In [ ]:
#@title
%%html
<marquee style='width: 30%; font-size: 30px; font-family: cursive; color: black;'><b>Next cell only for yearly data</b></marquee>

In [ ]:
df.rename(columns={'Station': 'Process', 'Serial Number': 'SerialNumber', 'Defect Status': 'DefectStatus'}, inplace=True)

In [ ]:
#@title Quadrant Column
# Function to create a column which contains info about the type of Film
# Plover or Quail, Shell or Ballonet

def film_type(data):
  if data['SerialNumber'][0] == 'P': # If the first char on Serial Number is 'P'
    if data['Location'][0] == 'B' : # If the first char on Reference is 'B'
      return 'PB' # Plover Ballonet
    else:
      return 'PS' # Plover Shell
  else: # Else it must be Quail
    if data['Location'][0] == 'B': # If the first char on Reference is 'B'
      return 'QB' # Quail Ballonet
    else:
      return 'QS' # Quail... Shell

# Create Type column for function above
df['Type'] = df.apply(film_type, axis=1)

In [ ]:
#@title Data Cleanup & Prep

# Remove defects which are Deleted in order to only keep Closed defects
df = df.drop(df[df['DefectStatus'] == 'Deleted'].index)

df['Operation'] = df['Operation'].str.upper()
df['Process'] = df['Process'].str.upper()
df['SubProcess'] = df['SubProcess'].str.upper()
df['Condition'] = df['Condition'].str.upper()
df['Disposition'] = df['Disposition'].str.upper()
df['Comment'] = df['Comment'].str.lower()


df['Operation'] = df['Operation'].str.strip()
df['Process'] = df['Process'].str.strip()
df['SubProcess'] = df['SubProcess'].str.strip()
df['Condition'] = df['Condition'].str.strip()
df['Disposition'] = df['Disposition'].str.strip()

balloons = df['SerialNumber'].unique().tolist()
balloons.sort()

In [ ]:
#@title Grading Standard
# If Standard, the Standard limits apply, else... 
threshold = 'Standard' #@param ["Standard", "Strict"] {allow-input: false}

# The following dicts contain the limits (n) and balloon rating (r) [n, r]
# for every group on both shell and ballonet
# NO_COMMENTS limits are up to my own discretion since no limits were provided 

PLOVER_LIMITS_STANDARD = {'S': {'RESEAL_CUTOFF1': [9, 'GT'],
                                'CUTOFF': [3, 'GT'], 'TUCK2': [11, 'GT'],
                                'BALLOON_UNPACKING2': [0, 'GT'],
                                'HOLE_IN_SEAL': [0, 'GT'],
                                'TACK_POINT': [0, 'GT'], 'SNOWBALL': [0, 'GT'],
                                'NO_COMMENTS': [14, 'GT'],
                                'RETURN_TO_SEALING': [0, 'GT'],
                                'REMOVE_FEATURE_PLATE': [0, 'GT'],
                                'DELTA': [-3, 'OK', 'PT']},
                          'B': {'RESEAL_CUTOFF1': [8, 'PT'],
                                'CUTOFF': [2, 'PT'], 'TUCK2': [13, 'PT'],
                                'BALLOON_UNPACKING2': [0, 'GT'],
                                'HOLE_IN_SEAL': [0, 'PT'],
                                'TACK_POINT': [0, 'GT'], 'SNOWBALL': [0, 'PT'],
                                'NO_COMMENTS': [9, 'PT'],
                                'RETURN_TO_SEALING': [0, 'GT'],
                                'REMOVE_FEATURE_PLATE': [0, 'GT']}}


QUAIL_LIMITS_STANDARD = {'S': {'RESEAL_CUTOFF1': [13, 'GT'],
                               'CUTOFF': [5, 'GT'], 'TUCK2': [10, 'GT'],
                               'BALLOON_UNPACKING2': [0, 'GT'],
                               'HOLE_IN_SEAL': [0, 'GT'],
                               'TACK_POINT': [0, 'GT'], 'SNOWBALL': [0, 'GT'],
                               'NO_COMMENTS': [19, 'GT'],
                               'RETURN_TO_SEALING': [0, 'GT'],
                               'REMOVE_FEATURE_PLATE': [0, 'GT'],
                               'DELTA': [-3, 'OK', 'PT']},
                         'B': {'RESEAL_CUTOFF1': [15, 'PT'],
                               'CUTOFF': [5, 'PT'], 'TUCK2': [24, 'PT'],
                               'BALLOON_UNPACKING2': [0, 'GT'],
                               'HOLE_IN_SEAL': [0, 'PT'],
                               'TACK_POINT': [0, 'GT'], 'SNOWBALL': [0, 'PT'],
                               'NO_COMMENTS': [14, 'PT'],
                               'RETURN_TO_SEALING': [0, 'GT'],
                               'REMOVE_FEATURE_PLATE': [0, 'GT']}}

PLOVER_LIMITS_STRICT = {'S': {'RESEAL_CUTOFF1': [9, 'GT'],
                              'CUTOFF': [3, 'GT'], 'TUCK2': [11, 'GT'],
                              'BALLOON_UNPACKING2': [0, 'GT'],
                              'HOLE_IN_SEAL': [0, 'GT'],
                              'TACK_POINT': [0, 'GT'], 'SNOWBALL': [0, 'GT'],
                              'NO_COMMENTS': [9, 'GT'],
                              'RETURN_TO_SEALING': [0, 'GT'],
                              'REMOVE_FEATURE_PLATE': [0, 'GT'],
                              'DELTA': [-3, 'OK', 'PT']},
                        'B': {'RESEAL_CUTOFF1': [4, 'PT'],
                              'CUTOFF': [0, 'PT'], 'TUCK2': [6, 'PT'],
                              'BALLOON_UNPACKING2': [0, 'GT'],
                              'HOLE_IN_SEAL': [0, 'PT'],
                              'TACK_POINT': [0, 'GT'], 'SNOWBALL': [0, 'PT'],
                              'NO_COMMENTS': [4, 'PT'],
                              'RETURN_TO_SEALING': [0, 'GT'],
                              'REMOVE_FEATURE_PLATE': [0, 'GT']}}


QUAIL_LIMITS_STRICT = {'S': {'RESEAL_CUTOFF1': [13, 'GT'],
                             'CUTOFF': [5, 'GT'], 'TUCK2': [10, 'GT'],
                             'BALLOON_UNPACKING2': [0, 'GT'],
                             'HOLE_IN_SEAL': [0, 'GT'],
                             'TACK_POINT': [0, 'GT'], 'SNOWBALL': [0, 'GT'],
                             'NO_COMMENTS': [14, 'GT'],
                             'RETURN_TO_SEALING': [0, 'GT'],
                             'REMOVE_FEATURE_PLATE': [0, 'GT'],
                             'DELTA': [-3, 'OK', 'PT']},
                       'B': {'RESEAL_CUTOFF1': [4, 'PT'], 'CUTOFF': [0, 'PT'],
                             'TUCK2': [13, 'PT'], 
                             'BALLOON_UNPACKING2': [0, 'GT'],
                             'HOLE_IN_SEAL': [0, 'PT'], 'TACK_POINT': [0, 'GT'],
                             'SNOWBALL': [0, 'PT'], 'NO_COMMENTS': [9, 'GT'],
                             'RETURN_TO_SEALING': [0, 'GT'],
                             'REMOVE_FEATURE_PLATE': [0, 'GT']}}

PLOVER_LIMITS = PLOVER_LIMITS_STANDARD if threshold == 'Standard' else PLOVER_LIMITS_STRICT
QUAIL_LIMITS = QUAIL_LIMITS_STANDARD if threshold == 'Standard' else QUAIL_LIMITS_STRICT

In [ ]:
#@title Groups Criteria

# Manual Groups

RETURN_TO_SEALING = {'Process': ('FINISHING TABLE', 'VERTROD'),
                     'SubProcess': ('CLOSING VERTROD', 'GORE FILM',
                                    'GORE SEALS', 'TACK MARKING'),
                     'Condition': ('CONTAMINATION', 'EXTRA GORE', 'HOLE', 
                                   'HOLE IN SEAL', 'MARKING', 'RIP/TEAR/CUT',
                                   'RIP/TEAR/YIELDED EDGE', 'SCRATCH/ABRASION',
                                   'SEALED EXTRA MATERIAL'),
                     'Disposition': ('MRB',)}

REMOVE_FEATURE_PLATE = {'Process': ('FINISHING TABLE',),
                        'SubProcess': ('DONUT SEAL',),
                        'Condition': ('BALLONET CLOCKING', 'CONTAMINATION',
                                      'FEATURE PLATE CLOCKING', 'HOLE',
                                      'SEALED EXTRA MATERIAL'),
                        'Disposition': ('MRB',)}
       
BALLON_UNPACKING = {'Process': ('FINISHING TABLE', 'TESTING'),
                    'SubProcess': ('ELECTRONICS', 'TENDON'),
                    'Condition': ('BROKEN CONNECTOR', 'DAMAGE', 'TACK DOT'),
                    'Disposition': ('REPLACE CABLE',
                                    'REPLACE WITH LIKE TENDON MFG METHOD; TENDON RETHREAD',
                                    'REPLACE; TENDON RETHREAD')}

TUCK = {'Process': ('FINISHING TABLE', 'VERTROD'),
        'SubProcess': ('CLOSING VERTROD', 'GORE SEALS'),
        'Condition': ('CURVED VERTROD', 'FLIPPED BARRIER LAYER', 'SEAL DEFECT'),
        'Disposition': ('MRB',)}

RESEAL_CUTOFF = {'Process': ('0', '1', '2', '3', '4', '5', '6',
                             'FINISHING TABLE', 'SEALING TABLE 1',
                             'SEALING TABLE 2', 'SEALING TABLE 3',
                             'SEALING TABLE 4', 'SEALING TABLE 5',
                             'SEALING TABLE 6', 'VERTROD'),
                 'SubProcess': ('BALLONET FLAGS', 'CLOSING VERTROD',
                                'GORE FILM', 'GORE SEALS', 'TACK MARKING',
                                'TENDON SLEEVE'),
                 'Condition': ('ABNORMALITY', 'DIMENSIONAL', 'EXTRA GORE',
                               'FINGER PULL OR YIELDING', 'LOOKS DIFFERENT',
                               'MISALIGNED CENTERFOLD', 'OTHER', 'PEELING SEAL',
                               'POSITION', 'PROGRESSION', 'RECIPE',
                               'SEALER ALARM', 'SEALER BAND', 'VERTROD STEP',
                               'WRONG CTAPE/BACKING', 'WRONG RECIPE'),
                 'Disposition': ('MRB', 'CUT OFF')}

SNOWBALL2 = {'Process': ('1', '2', '3', '4', '5', 'SEALING TABLE 1',
                        'SEALING TABLE 2', 'SEALING TABLE 3',
                        'SEALING TABLE 4', 'SEALING TABLE 5',
                        'SEALING TABLE 6'),
            'SubProcess': ('GORE FILM', 'GORE SEALS', 'SEAL ON SEAL'),
            'Condition': ('FAILED REPAIR', 'FAILED RESEAL', 'SEALER BAND'),
            'Disposition': ('CUT OFF', 'MRB')}



# List of manual groups
MANUAL = ['RETURN_TO_SEALING', 'REMOVE_FEATURE_PLATE', 'BALLON_UNPACKING',
          'TUCK', 'SNOWBALL2', 'RESEAL_CUTOFF']


# TACK_POINT, HOLE_IN_SEAL, and CUTOFF are not needed in manual_findings
# as they are not a manual task, however they are included here in order to be 
# able to sum the qtys here to the qtys in balloon_defects dict without getting
# a KeyError: 'HOLE_IN_SEAL'

manual_findings = {i : {'S': {'RETURN_TO_SEALING': 0,
                        'REMOVE_FEATURE_PLATE': 0, 'BALLOON_UNPACKING2': 0,
                        'TUCK2': 0, 'RESEAL_CUTOFF1': 0, 'NO_COMMENTS': 0,
                        'TACK_POINT': 0, 'SNOWBALL': 0, 'HOLE_IN_SEAL': 0,
                        'CUTOFF': 0},
                        'B': {'RETURN_TO_SEALING': 0,
                        'REMOVE_FEATURE_PLATE': 0, 'BALLOON_UNPACKING2': 0,
                        'TUCK2': 0, 'RESEAL_CUTOFF1': 0, 'NO_COMMENTS': 0,
                        'TACK_POINT': 0, 'SNOWBALL': 0, 'HOLE_IN_SEAL': 0,
                        'CUTOFF': 0}} for i in df['SerialNumber'].unique()}


# Groups 1 & 2 Criteria

#### GROUP 2 ####

TUCK2 = {'Process': ('1', '2', '3', '4', '5', 'FINISHING TABLE',
                     'SEALING TABLE 1', 'SEALING TABLE 2', 'SEALING TABLE 3',
                     'SEALING TABLE 4', 'SEALING TABLE 5', 'SEALING TABLE 6'),
         'SubProcess': ('CLOSING VERTROD', 'GORE SEALS'),
         'Condition': ('MARKLINE TUCK', 'TUCK'),
         'Disposition': ('INCISION & RESEAL', 'MRB', 'RESEAL', 'SSA PATCH',
                         'SSA PATCH (APPLIED EXTERIOR)', 'VERTROD',
                         'VERTROD & SSA PATCH', 
                         'VERTROD & SSA PATCH (APPLIED EXTERIOR)')}


BALLOON_UNPACKING2 = {'Process': ('BALLOON PACKAGING', 'FINISHING TABLE'),
                     'SubProcess': ('ELECTRONICS', 'INSPECTION'),
                     'Condition': ('BALLOON UNPACKING', 'BROKEN CONNECTOR'),
                     'Disposition': ('MRB',)}


TACK_POINT = {'Process': ('FINISHING TABLE',),
              'SubProcess': ('TACK WINDOW',),
              'Condition': ('TORN WINDOW', 'WRONG POSITION', 'WRONG SIZE'),
              'Disposition': ('MRB',)}


SNOWBALL = {'Process': ('1', '2', '3', '4', '5', 'SEALING TABLE 1',
                         'SEALING TABLE 2', 'SEALING TABLE 3',
                         'SEALING TABLE 4', 'SEALING TABLE 5',
                         'SEALING TABLE 6'),
             'SubProcess': ('GORE FILM', 'GORE SEALS'),
             'Condition': ('FAILED RESEAL FINAL SEAL', 'FILM DAMAGE'),
             'Disposition': ('MRB',)}


HOLE_IN_SEAL = {'Process': ('1', '2', '3', '4', '5', 'SEALING TABLE 1',
                            'SEALING TABLE 2', 'SEALING TABLE 3',
                            'SEALING TABLE 4', 'SEALING TABLE 5',
                            'SEALING TABLE 6'),
                'SubProcess': ('GORE SEALS',),
                'Condition': ('HOLE IN SEAL',),
                'Disposition': ('VERTROD',)}


### GROUP 1 ###

# RESEAL by itself is not used, but left here in case it needs to be separated

RESEAL = {'Process': ('1', '2', '3', '4', '5', 'SEALING TABLE 1',
                      'SEALING TABLE 2', 'SEALING TABLE 3', 'SEALING TABLE 4',
                      'SEALING TABLE 5', 'SEALING TABLE 6', 'VERTROD'),
          'SubProcess': ('GORE SEALS', 'TENDON SLEEVE'),
          'Condition': ('CLEAR EDGE', 'CONTAMINATION', 'DAMAGE',
                        'DEEP KNIFE CUT', 'DEEP SCISSOR CUT',
                        'DEEP SEALER KNIFE CUT', 'FAILED REPAIR',
                        'FILM DAMAGE', 'FLIPPED BARRIER LAYER', 
                        'HOLE IN SEAL', 'JAGGED EDGE',
                        'REPEATING DEFECT', 'SEAL BUBBLES',
                        'SEAL NARROWING', 'SEALED EXTRA MATERIAL',
                        'SEALING INTERRUPTION', 'SPLICE IN SEAL',
                        'TUCK', 'VOID CTAPE/BACKING',
                        'VOID GORE MATERIAL','YIELDED SEAL'),
          'Disposition': ('CHANGE BANDS AND RESEAL', 'CLEAN AND RESEAL',
                          'INCISION & RESEAL', 'RESEAL',
                          'SEALER VALIDATION & RESEAL')}


CUTOFF = {'Process': ('1', '2', '3', '4', '5', 'SEALING TABLE 1',
                      'SEALING TABLE 2', 'SEALING TABLE 3', 'SEALING TABLE 4',
                      'SEALING TABLE 5', 'SEALING TABLE 6', 'VERTROD'),
          'SubProcess': ('GORE FILM', 'GORE SEALS'),
          'Condition': ('CONTAMINATION', 'DEEP KNIFE CUT', 'DEEP SCISSOR CUT',
                        'DEEP SEALER KNIFE CUT', 'FAILED REPAIR',
                        'FAILED RESEAL', 'FILM DAMAGE', 'GORE PROGRESSION',
                        'HOLE', 'SCRATCH/ABRASION', 'SEAL ON SEAL',
                        'SEALED EXTRA MATERIAL', 'SEALING INTERRUPTION',
                        'YIELDED SEAL'),
          'Disposition': ('CUT OFF', 'SEALER VALIDATION & RESEAL')}

RESEAL_CUTOFF1 = {'Process': ('1', '2', '3', '4', '5', 'SEALING TABLE 1',
                              'SEALING TABLE 2', 'SEALING TABLE 3',
                              'SEALING TABLE 4', 'SEALING TABLE 5',
                              'SEALING TABLE 6', 'VERTROD'),
                  'SubProcess': ('GORE FILM', 'GORE SEALS', 'TENDON SLEEVE'),
                  'Condition': ('CLEAR EDGE', 'CONTAMINATION', 'DAMAGE',
                                'DEEP SCISSOR CUT', 'DEEP SEALER KNIFE CUT',
                                'FAILED REPAIR', 'FAILED RESEAL', 'FILM DAMAGE',
                                'FLIPPED BARRIER LAYER', 'GORE PROGRESSION',
                                'HOLE', 'HOLE IN SEAL', 'JAGGED EDGE',
                                'REPEATING DEFECT', 'SCRATCH/ABRASION',
                                'SEAL BUBBLES', 'SEAL NARROWING',
                                'SEAL ON SEAL', 'SEALED EXTRA MATERIAL',
                                'SEALING INTERRUPTION', 'SPLICE IN SEAL',
                                'TUCK', 'VOID CTAPE/BACKING',
                                'VOID GORE MATERIAL', 'YIELDED SEAL'),
                  'Disposition': ('CHANGE BANDS AND RESEAL', 'CLEAN AND RESEAL',
                                  'CUT OFF', 'INCISION & RESEAL', 'RESEAL',
                                  'SEALER VALIDATION & CUT OFF',
                                  'SEALER VALIDATION & RESEAL')}

In [ ]:
#@title Balloons in this dataset
df['SerialNumber'].unique()

array(['P-01056', 'P-01057', 'P-01058', 'P-01059', 'P-01060', 'P-01061',
       'P-01062', 'P-01063', 'P-01064', 'Q-00221', 'Q-00222', 'Q-00223',
       'Q-00224', 'Q-00225', 'Q-00226', 'Q-00227', 'Q-00228'],
      dtype=object)

In [ ]:
#@title Specific Balloon Data
balloon_name = 'Q-00222' #@param {type:"string"}
df[df['SerialNumber'] == balloon_name]

In [ ]:
#@title Manual Screening
group_selector = widgets.SelectMultiple(options=MANUAL,
                                       value=MANUAL,
                                       rows=len(MANUAL),
                                       description='Manual',
                                       disabled=False)
display(group_selector)

SelectMultiple(description='Manual', index=(0, 1, 2, 3, 4, 5), options=('RETURN_TO_SEALING', 'REMOVE_FEATURE_P…

In [ ]:
#@title Manual Review

quail_shell = df.loc[(df['Process'].isin(eval(group_selector.value[0])['Process'])) &
                     (df['SubProcess'].isin(eval(group_selector.value[0])['SubProcess'])) &
                     (df['Condition'].isin(eval(group_selector.value[0])['Condition'])) &
                     (df['Disposition'].isin(eval(group_selector.value[0])['Disposition'])) &
                     (df['Type'] == 'QS')]

quail_ballonet = df.loc[(df['Process'].isin(eval(group_selector.value[0])['Process'])) &
                        (df['SubProcess'].isin(eval(group_selector.value[0])['SubProcess'])) &
                        (df['Condition'].isin(eval(group_selector.value[0])['Condition'])) &
                        (df['Disposition'].isin(eval(group_selector.value[0])['Disposition'])) &
                        (df['Type'] == 'QB')]
plover_shell = df.loc[(df['Process'].isin(eval(group_selector.value[0])['Process'])) &
                      (df['SubProcess'].isin(eval(group_selector.value[0])['SubProcess'])) &
                      (df['Condition'].isin(eval(group_selector.value[0])['Condition'])) &
                      (df['Disposition'].isin(eval(group_selector.value[0])['Disposition'])) &
                      (df['Type'] == 'PS')]

plover_ballonet = df.loc[(df['Process'].isin(eval(group_selector.value[0])['Process'])) &
                         (df['SubProcess'].isin(eval(group_selector.value[0])['SubProcess'])) &
                         (df['Condition'].isin(eval(group_selector.value[0])['Condition'])) &
                         (df['Disposition'].isin(eval(group_selector.value[0])['Disposition'])) &
                         (df['Type'] == 'PB')]

quadrants = [quail_shell, quail_ballonet, plover_shell, plover_ballonet]
lines = pd.concat(quadrants)
lines[['SerialNumber', 'Process', 'Operation', 'SubProcess', 'Condition',
       'Criteria',	'Disposition', 'Comment', 'Location', 'Type']]


,SerialNumber,Process,Operation,SubProcess,Condition,Criteria,Disposition,Comment,Location,Type
258,Q-00222,FINISHING TABLE,FINISHING,DONUT SEAL,CONTAMINATION,PRESENCE OF INK THAT VIOLATES THE 1/16 IN MIMIMUM SEAL REQUIREMENT OF EITHER EDGE OF THE SEAL,MRB,small ink dot; use as is,12_42A,QS
386,Q-00228,FINISHING TABLE,FINISHING,DONUT SEAL,CONTAMINATION,PRESENCE OF INK THAT VIOLATES THE 1/16 IN MIMIMUM SEAL REQUIREMENT OF EITHER EDGE OF THE SEAL,MRB,"the operator did not orient the markline properly on the first seal and did not recogize it until after. will cut off the doubler only, resituate the shell markline to the pedestal properly, apply a new doubler and seal as intended.",16_46A,QS


In [ ]:
#@title Fetch Comments. Use | for or e.g. tuck|vertrod
pattern = 'tuck|vertrod' #@param {type:"string"}

df[df['Comment'].str.contains(pattern)]

In [ ]:
#@title Manual Entries

serial_number = widgets.Dropdown(options=balloons,)

rts = widgets.Box([widgets.Label(value='RETURN_TO_SEALING'),
                   widgets.IntText(value=0, description='Qty:',
                                   disabled=False),
             widgets.RadioButtons(options=['S', 'B'])],
                  layout={'width': 'max-content'})

rfp = widgets.Box([widgets.Label(value='REMOVE_FEATURE_PLATE'),
                   widgets.IntText(value=0, description='Qty:',
                                   disabled=False),
             widgets.RadioButtons(options=['S', 'B'])],
                  layout={'width': 'max-content'})

bu = widgets.Box([widgets.Label(value='BALLOON_UNPACKING2'),
                   widgets.IntText(value=0, description='Qty:',
                                   disabled=False),
             widgets.RadioButtons(options=['S', 'B'])],
                  layout={'width': 'max-content'})

tuck = widgets.Box([widgets.Label(value='TUCK2'),
                   widgets.IntText(value=0, description='Qty:',
                                   disabled=False),
             widgets.RadioButtons(options=['S', 'B'])],
                  layout={'width': 'max-content'})

snowball = widgets.Box([widgets.Label(value='SNOWBALL'),
                   widgets.IntText(value=0, description='Qty:',
                                   disabled=False),
             widgets.RadioButtons(options=['S', 'B'])],
                  layout={'width': 'max-content'})

reseal_cutoff = widgets.Box([widgets.Label(value='RESEAL_CUTOFF1'),
                   widgets.IntText(value=0, description='Qty:',
                                   disabled=False),
             widgets.RadioButtons(options=['S', 'B'])],
                  layout={'width': 'max-content'})

no_comments = widgets.Box([widgets.Label(value='NO_COMMENTS'),
                   widgets.IntText(value=0, description='Qty:',
                                   disabled=False),
             widgets.RadioButtons(options=['S', 'B'])],
                  layout={'width': 'max-content'})

button = widgets.Button(description='Save', tooltip='Apply Changes')


def balloon_status(button):
  for i in [rts, rfp, bu, tuck, snowball, reseal_cutoff, no_comments]:
    manual_findings[serial_number.value][i.children[2].value][i.children[0].value] += i.children[1].value
    i.children[1].value = 0
  print(serial_number.value, ':')
  pprint.pprint(manual_findings[serial_number.value])

display(serial_number, rts, rfp, bu, tuck, snowball, reseal_cutoff, no_comments,
        button)

button.on_click(balloon_status)

Dropdown(options=('P-01056', 'P-01057', 'P-01058', 'P-01059', 'P-01060', 'P-01061', 'P-01062', 'P-01063', 'P-0…

Box(children=(Label(value='RETURN_TO_SEALING'), IntText(value=0, description='Qty:'), RadioButtons(options=('S…

Box(children=(Label(value='REMOVE_FEATURE_PLATE'), IntText(value=0, description='Qty:'), RadioButtons(options=…

Box(children=(Label(value='BALLOON_UNPACKING2'), IntText(value=0, description='Qty:'), RadioButtons(options=('…

Box(children=(Label(value='TUCK2'), IntText(value=0, description='Qty:'), RadioButtons(options=('S', 'B'), val…

Box(children=(Label(value='SNOWBALL'), IntText(value=0, description='Qty:'), RadioButtons(options=('S', 'B'), …

Box(children=(Label(value='RESEAL_CUTOFF1'), IntText(value=0, description='Qty:'), RadioButtons(options=('S', …

Box(children=(Label(value='NO_COMMENTS'), IntText(value=0, description='Qty:'), RadioButtons(options=('S', 'B'…

Button(description='Save', style=ButtonStyle(), tooltip='Apply Changes')

In [ ]:
#@title Groups Processing

GROUPS = ['TUCK2', 'BALLOON_UNPACKING2', 'TACK_POINT', 'SNOWBALL',
          'HOLE_IN_SEAL', 'RESEAL_CUTOFF1', 'CUTOFF', 'RETURN_TO_SEALING']

balloon_defects = {balloon: {'S': {'TUCK2': 0, 'BALLOON_UNPACKING2': 0,
                                   'TACK_POINT': 0, 'SNOWBALL': 0,
                                   'HOLE_IN_SEAL': 0, 'RESEAL_CUTOFF1': 0,
                                   'CUTOFF': 0, 'RETURN_TO_SEALING': 0,
                                   'REMOVE_FEATURE_PLATE': 0},
                             'B': {'TUCK2': 0, 'BALLOON_UNPACKING2': 0,
                                   'TACK_POINT': 0, 'SNOWBALL': 0,
                                   'HOLE_IN_SEAL': 0, 'RESEAL_CUTOFF1': 0,
                                   'CUTOFF': 0, 'RETURN_TO_SEALING': 0,
                                   'REMOVE_FEATURE_PLATE': 0}} for balloon in df['SerialNumber'].unique()}

# These two loops are O(n3), but the slowdown is from the isin function which is slow
for b in df['SerialNumber'].unique():
  for q in ['S', 'B']: # Shell or Ballonet
    for group in GROUPS:
      balloon_defects[b][q][group] = df.loc[(df['SerialNumber'] == b) &
                                         (df['Type'].str[1] == q) & (df['Process'].isin(eval(group)['Process'])) &
                                         (df['SubProcess'].isin(eval(group)['SubProcess'])) &
                                         (df['Condition'].isin(eval(group)['Condition'])) &
                                         (df['Disposition'].isin(eval(group)['Disposition']))]['SerialNumber'].count()


# Combine Manual Groups (3 & 4) with Groups Processed by code (1 & 2)  
for b in balloon_defects: # For every balloon
  for f in balloon_defects[b]: # For every film (shell, ballonet)
    for d in balloon_defects[b][f]: # For every defect/group
      # Add the manual_findings qtys to the balloon_defects qtys essentially
      # summing up what the code found plus what the user found manually
      balloon_defects[b][f][d] += manual_findings[b][f][d]

In [ ]:
#@title Balloon Details

balloon_details = widgets.Dropdown(options=balloons,)
details_button = widgets.Button(description='Enter',
                                tooltip='Get Balloon Details')

def balloon_summary(details_button):
  print(balloon_details.value)
  pprint.pprint(balloon_defects[balloon_details.value])

display(balloon_details, details_button)

details_button.on_click(balloon_summary)


Dropdown(options=('P-01056', 'P-01057', 'P-01058', 'P-01059', 'P-01060', 'P-01061', 'P-01062', 'P-01063', 'P-0…

Button(description='Enter', style=ButtonStyle(), tooltip='Get Balloon Details')

Q-00222
{'B': {'BALLOON_UNPACKING2': 0,
       'CUTOFF': 0,
       'HOLE_IN_SEAL': 0,
       'REMOVE_FEATURE_PLATE': 0,
       'RESEAL_CUTOFF1': 0,
       'RETURN_TO_SEALING': 0,
       'SNOWBALL': 0,
       'TACK_POINT': 0,
       'TUCK2': 1},
 'S': {'BALLOON_UNPACKING2': 0,
       'CUTOFF': 0,
       'HOLE_IN_SEAL': 0,
       'REMOVE_FEATURE_PLATE': 1,
       'RESEAL_CUTOFF1': 0,
       'RETURN_TO_SEALING': 0,
       'SNOWBALL': 0,
       'TACK_POINT': 0,
       'TUCK2': 0}}


In [ ]:
#@title Delta Calculation
balloon_delta = {balloon: 0 for balloon in df['SerialNumber'].unique()}

for b in balloon_defects:
  balloon_delta[b] += (balloon_defects[b]['S']['RESEAL_CUTOFF1'] - balloon_defects[b]['B']['RESEAL_CUTOFF1'])

In [ ]:
#@title Defects Overview
data = pd.DataFrame.from_dict(balloon_defects, orient='index').stack().to_frame()
data = pd.DataFrame(data[0].values.tolist(), index=data.index)
data['DELTA'] = data.index.get_level_values(level=0).map(balloon_delta)
data

In [ ]:
#@title Add Month Data to Output

df['DateCreated'] = pd.to_datetime(df['DateCreated'])
df['Month'] = df['DateCreated'].dt.strftime('%B %Y')

balloon_month = dict(df.groupby(['SerialNumber']).apply(lambda x: x['Month'].unique()).apply(pd.Series)[0])
data['Month'] = data.index.get_level_values(level=0).map(balloon_month)

In [ ]:
#@title Manual Findings Summary
manual = pd.DataFrame.from_dict(manual_findings, orient='index').stack().to_frame()
manual = pd.DataFrame(manual[0].values.tolist(), index=manual.index)
manual.replace(np.nan, 0, inplace=True)
manual.sort_index(inplace=True)

# manual = manual.reset_index()
manual.rename(columns={'RETURN_TO_SEALING': 'Returned to Sealing Table',
                       'REMOVE_FEATURE_PLATE': 'Feature Plate Removal',
                       'BALLOON_UNPACKING2': 'Balloon Unpacking',
                       'TUCK2': 'Hidden Tucks',
                       'RESEAL_CUTOFF1': 'Reseals or Cutoffs',
                       'NO_COMMENTS': 'No Comments', 'TACK_POINT': 'Tack Point',
                       'SNOWBALL': 'Snowball', 'HOLE_IN_SEAL': 'Hole in Seal',
                       'CUTOFF': 'Cut offs'}, inplace=True)
manual

In [ ]:
#@title Status Grading
grades = {b: {'S': {'TUCK2':'OK', 'BALLOON_UNPACKING2':'OK', 'TACK_POINT':'OK',
              'SNOWBALL':'OK', 'HOLE_IN_SEAL':'OK', 'RESEAL_CUTOFF1':'OK',
              'CUTOFF':'OK', 'RETURN_TO_SEALING': 'OK',
              'REMOVE_FEATURE_PLATE': 'OK', 'DELTA': 'OK'},
          'B': {'TUCK2':'OK', 'BALLOON_UNPACKING2':'OK', 'TACK_POINT':'OK',
              'SNOWBALL':'OK', 'HOLE_IN_SEAL':'OK', 'RESEAL_CUTOFF1':'OK',
              'CUTOFF':'OK', 'RETURN_TO_SEALING': 'OK',
              'REMOVE_FEATURE_PLATE': 'OK', 'DELTA': 'OK' }} for b in df['SerialNumber'].unique()}


defects = ['TUCK2', 'BALLOON_UNPACKING2', 'TACK_POINT', 'SNOWBALL',
           'HOLE_IN_SEAL', 'CUTOFF', 'RETURN_TO_SEALING',
           'REMOVE_FEATURE_PLATE']

# RESEAL_CUTOFF1 is not graded with Delta

for i in data.itertuples():
  if i.Index[0].startswith('P'):
    if i.Index[1] == 'S':
      if getattr(i, 'DELTA') >= PLOVER_LIMITS['S']['DELTA'][0]: # Delta > -3
        grades[i[0][0]]['S']['DELTA'] = PLOVER_LIMITS['S']['DELTA'][1] # Grade = 'OK'
      elif getattr(i, 'DELTA') < PLOVER_LIMITS['S']['DELTA'][0]: # Delta < -3
        grades[i[0][0]]['S']['DELTA'] = PLOVER_LIMITS['S']['DELTA'][2] # Grade = 'PT'

      for d in defects:
        if getattr(i, d) > PLOVER_LIMITS['S'][d][0]:
          grades[i[0][0]]['S'][d] = PLOVER_LIMITS['S'][d][1]
    else:
      for d in defects:
        if getattr(i, d) > PLOVER_LIMITS['B'][d][0]:
          grades[i[0][0]]['B'][d] = PLOVER_LIMITS['B'][d][1]
  else:
    if i.Index[1] == 'S':
      for d in defects:
        if getattr(i, d) > QUAIL_LIMITS['S'][d][0]:
          grades[i[0][0]]['S'][d] = QUAIL_LIMITS['S'][d][1]
    else:
      for d in defects:
        if getattr(i, d) > QUAIL_LIMITS['B'][d][0]:
          grades[i[0][0]]['B'][d] = QUAIL_LIMITS['B'][d][1]

In [ ]:
#@title Balloon Statuses

def _color_if_not_ok(s):
    return ['background-color: yellow' if val not in ['OK', 0] else '' for val in s]

result = pd.DataFrame.from_dict(grades, orient='index').stack().to_frame()
result = pd.DataFrame(result[0].values.tolist(), index=result.index)

result.sort_index(inplace=True)
# result = result.reset_index()
# result.rename(columns={'level_0': 'Balloon', 'level_1': 'Type'}, inplace=True)

# 'RESEAL_CUTOFF1' removed from below
result['GT Reasons'] = (result[['TUCK2', 'BALLOON_UNPACKING2', 'TACK_POINT',
                                'SNOWBALL', 'HOLE_IN_SEAL',
                                'CUTOFF', 'RETURN_TO_SEALING',
                                'REMOVE_FEATURE_PLATE', 'DELTA']] == 'GT').sum(axis=1)

result['PT Reasons'] = (result[['TUCK2', 'BALLOON_UNPACKING2', 'TACK_POINT',
                                'SNOWBALL', 'HOLE_IN_SEAL',
                                'CUTOFF', 'RETURN_TO_SEALING',
                                'REMOVE_FEATURE_PLATE', 'DELTA']] == 'PT').sum(axis=1)
result1 = result.copy(deep=True)
result

In [ ]:
#@title Highlight Ratings
result.style.apply(_color_if_not_ok)

In [ ]:
#@title Full Details
final = pd.concat([data, manual], axis=1, sort=False)
final.replace(np.nan, 0, inplace=True)

final.reset_index(inplace=True)
final.rename(columns={'level_0': 'Balloon', 'level_1': 'Film'}, inplace=True)
final

In [ ]:
#@title Balloons Grades
result1.reset_index(inplace=True)
result1.rename(columns={'level_0': 'Balloon', 'level_1': 'Type'}, inplace=True)

grounded = result1.groupby(level=0).filter(lambda x : x['GT Reasons'].gt(0).all())['Balloon']
protos = result1.groupby(level=0).filter(lambda x : x['PT Reasons'].gt(0).all())['Balloon']
okay = result1.groupby(level=0).filter(lambda x : x['GT Reasons'].lt(1).all())['Balloon']

true_protos = pd.Series(np.setdiff1d(protos.values,grounded.values))
semi_okay = pd.Series(np.setdiff1d(okay.values,grounded.values))
true_okay = pd.Series(np.setdiff1d(semi_okay.values,protos.values))

overall_grades = pd.DataFrame({'OK': pd.Series(true_okay), 'Proto': pd.Series(true_protos), 'Ground Test': pd.Series(grounded)}).reset_index(drop=True).fillna('--').drop_duplicates()
overall_grades
# When a balloon appears twice on Proto or Ground Test, it is because both shell and ballonet met the criteria for that grade

In [ ]:
#@title Ground Test per Category
result['Quadrant'] = ''
result.reset_index(inplace=True)

def quad(row):
  if row['level_0'][0] == 'P': # If the first char on level_0 is 'P'
    if row['level_1'][0] == 'B' : # If the first char on level_1 is 'B'
      return 'PB' # Plover Ballonet
    else:
      return 'PS' # Plover Shell
  else: # Else it must be Quail
    if row['level_1'][0] == 'B' : # If the first char on level_0 is 'B'
      return 'QB' # Quail Ballonet
    else:
      return 'QS' # Quail Shell

# Create Type column for function above
result['Quadrant'] = result.apply(quad, axis=1)

# GT per quadrant per category

gts = result.groupby('Quadrant')[['CUTOFF', 'HOLE_IN_SEAL', 'BALLOON_UNPACKING2',
                                  'TACK_POINT', 'SNOWBALL', 'RETURN_TO_SEALING',
                                  'REMOVE_FEATURE_PLATE',
                                  'TUCK2', 'DELTA']].apply(lambda x: (x=='GT').sum()).T

gts

In [ ]:
#@title Proto per category

pts = result.groupby('Quadrant')[['CUTOFF', 'HOLE_IN_SEAL', 'BALLOON_UNPACKING2',
                                  'TACK_POINT', 'SNOWBALL', 'RETURN_TO_SEALING',
                                  'REMOVE_FEATURE_PLATE',
                                  'TUCK2', 'DELTA']].apply(lambda x: (x=='PT').sum()).T

pts

In [ ]:
#@title Output to Google Drive

standard = ': Delta '

workbook = gc.create(file_path + standard + 'grades')
spreadsheet = gc.open(file_path + standard + 'grades')

grades = spreadsheet.add_worksheet('Summary', rows=30, cols=10)
grading = spreadsheet.add_worksheet('Pass/No Pass', rows=150, cols=10)
details = spreadsheet.add_worksheet('Details', rows=150, cols=16)
pt_quadrants = spreadsheet.add_worksheet('Protos', rows=8, cols=11)
gt_quadrants = spreadsheet.add_worksheet('Ground Tests', rows=8, cols=11)
workbook.del_worksheet(workbook.sheet1)

spreadsheet.values_update

set_with_dataframe(grades, overall_grades, include_index=False)
set_with_dataframe(grading, result, include_index=False)
set_with_dataframe(details, final, include_index=False)
set_with_dataframe(pt_quadrants, pts, include_index=True)
set_with_dataframe(gt_quadrants, gts, include_index=True)